In [2]:
from ultralytics import YOLO
import cv2
from sort import*
import cvzone
import math

In [3]:
mymodel = YOLO('yolov8m.pt')




In [4]:

coco_classes = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
    'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


In [5]:
import cv2
import numpy as np
import math
from sort import Sort  
import cvzone

cap = cv2.VideoCapture("road2.mp4")

cap.set(3, 1280)  
cap.set(4, 720)  


mask = cv2.imread("mask.png")
mask = cv2.resize(mask, (1280, 720))  

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Video kaydetme
# output_path = "output_video_9.mp4"
# fourcc = cv2.VideoWriter_fourcc(*'XVID')
# out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


mask = cv2.imread("mask.png")
mask = cv2.resize(mask, (frame_width, frame_height))


max_age = 15
min_hits = 1
iou_threshold = 0.3
tracker = Sort(max_age=max_age, min_hits=min_hits, iou_threshold=iou_threshold)


lim =[230, 332,514, 324]
lim2 = [546, 320,754, 316]
lim3 = [842, 364,1134, 576]


cars=[]
cars1=[]
cars2=[]

while True:
    ret, frame = cap.read()

    if not ret:
        break

    imgRegion = cv2.bitwise_and(frame, mask)
    results = mymodel(imgRegion, stream=True)

    detects=np.empty((0,5))
   
    for r in results:
        boxes =r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            
            conf = math.ceil((box.conf[0]*100))/100
            cls = int(box.cls[0])
            currentClass = coco_classes[cls]
            
            
            if currentClass == "car" or  currentClass == "truck" or\
            currentClass =="bus" or currentClass == "motorcycle"and conf > 0.5:
                
                cArray = np.array([x1,y1,x2,y2,conf])
                detects = np.vstack((detects,cArray))

      
    confThreshold = 0.3
    nmsThreshold = 0.5
    indices = cv2.dnn.NMSBoxes(detects[:, :4], detects[:, 4], confThreshold, nmsThreshold)
    filtered_detects = detects[indices]       
   
    resultsTracker = tracker.update(filtered_detects)
    cv2.line(frame,(lim[0],lim[1]),(lim[2],lim[3]),(200,0,100),2)
    cv2.line(frame,(lim2[0],lim2[1]),(lim2[2],lim2[3]),(200,0,100),2)
    cv2.line(frame,(lim3[0],lim3[1]),(lim3[2],lim3[3]),(200,0,100),2)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id =result
        x1,y1,x2,y2  = int(x1),int(y1),int(x2),int(y2)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
        w,h = x2-x1,y2-y1
        cx,cy = x1+w//2,y1+h//2
        cv2.circle(frame,(int(cx),int(cy)),2,(255,0,255),cv2.FILLED)
        


       
        if lim[0]< cx <lim[2] and lim[1]- 15 <cy< lim[2] + 15:
            if cars.count(id)==0 or len(cars)==0:
                cars.append(id)
                
        if id in cars:
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
        
        
        
        if lim2[0]< cx <lim2[2] and lim2[1]- 15 <cy< lim2[2] + 15:
            if cars1.count(id)==0 or len(cars1)==0:
                cars1.append(id)
            
        if id in cars1:
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)  
        
        
        if lim3[0]< cx <lim3[2] and lim3[1]- 5 <cy< lim3[2] + 5:
            if cars2.count(id)==0 or len(cars2)==0:
                cars2.append(id)
                
        if id in cars2:
                    cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)     

            # cv2.putText(frame, f'Araba sayisi = {len(totalCount)}', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3 )
    cvzone.putTextRect(frame, f'Araba sayisi sol yol = {len(cars)}', (10,50),2,2)
    cvzone.putTextRect(frame, f'Araba sayisi orta yol = {len(cars1)}',(400,100),2,2)
    cvzone.putTextRect(frame, f'Araba sayisi sag yol = {len(cars2)}', (850,50),2,2)

    # out.write(frame)
    cv2.imshow("myframe",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
# out.release()




WARNING  NMS time limit 0.550s exceeded
0: 384x640 6 cars, 158.4ms
Speed: 0.0ms preprocess, 158.4ms inference, 1945.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 268.2ms
Speed: 0.0ms preprocess, 268.2ms inference, 3.7ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 43.1ms
Speed: 0.0ms preprocess, 43.1ms inference, 8.3ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 41.7ms
Speed: 0.0ms preprocess, 41.7ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 44.3ms
Speed: 0.0ms preprocess, 44.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 39.4ms
Speed: 0.0ms preprocess, 39.4ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 42.1ms
Speed: 0.0ms preprocess, 42.1ms inference, 5.4ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 37.0ms
Speed: 1.5ms preprocess, 37.0ms inference, 0.0ms postproce

: 

In [6]:







from ultralytics import YOLO
import cv2
from sort import*
import cvzone
import math
import time

mymodel = YOLO('yolov8m.pt')

coco_classes = [
    'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
    'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
    'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
    'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
    'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
    'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

cap = cv2.VideoCapture("road2.mp4")

cap.set(3, 1280)  
fps = int(cap.get(cv2.CAP_PROP_FPS))
cap.set(4, 720)  



mask = cv2.imread("mask.png")
mask = cv2.resize(mask, (1280, 720))  

tracker = Sort(max_age=18,min_hits=1, iou_threshold=0.3)




    
lim =[270, 220,496, 220]

lim2 = [499, 215,675, 203]
lim3 = [1029, 295,1257, 416]

lim4 = [150, 296,504, 300]

lim5 = [510, 300,731, 283]

lim6 = [857, 356,1133, 540]

mesafe = 9



cars=[]
cars1=[]
cars2=[]

cars_speed =[]
cars_speed1 = []
cars_speed2 = []


car_start_times = {}
car_start_times1 = {}
car_start_times2 = {}



frame_count = 0

while True:
    ret, frame = cap.read()
    frame_count += 1
    
    
    

    imgRegion = cv2.bitwise_and(frame, mask)
    results = mymodel(imgRegion, stream=True)

    detects=np.empty((0,5))
   
    for r in results:
        boxes =r.boxes
        for box in boxes:
            x1,y1,x2,y2 = box.xyxy[0]
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            
            conf = math.ceil((box.conf[0]*100))/100
            cls = int(box.cls[0])
            currentClass = coco_classes[cls]
            
            
            if currentClass == "car" or  currentClass == "truck" or\
            currentClass =="bus" or currentClass == "motorcycle"and conf > 0.5:
                
                cArray = np.array([x1,y1,x2,y2,conf])
                detects = np.vstack((detects,cArray))

      # Non-Maximum Suppression (NMS) 
    confThreshold = 0.3
    nmsThreshold = 0.5
    indices = cv2.dnn.NMSBoxes(detects[:, :4], detects[:, 4], confThreshold, nmsThreshold)
    filtered_detects = detects[indices]       
   
    resultsTracker = tracker.update(filtered_detects)
    cv2.line(frame,(lim[0],lim[1]),(lim[2],lim[3]),(200,0,100),2)
    cv2.line(frame,(lim2[0],lim2[1]),(lim2[2],lim2[3]),(200,0,100),2)
    cv2.line(frame,(lim3[0],lim3[1]),(lim3[2],lim3[3]),(200,0,100),2)

    cv2.line(frame,(lim4[0],lim4[1]),(lim4[2],lim4[3]),(200,0,100),2)
    cv2.line(frame,(lim5[0],lim5[1]),(lim5[2],lim5[3]),(200,0,100),2)
    cv2.line(frame,(lim6[0],lim6[1]),(lim6[2],lim6[3]),(200,0,100),2)
    
    
    
    
    
    
    
    for result in resultsTracker:
        x1,y1,x2,y2,id =result
        x1,y1,x2,y2  = int(x1),int(y1),int(x2),int(y2)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,0,255),2)
        w,h = x2-x1,y2-y1
        bx,by =  (x1 + x2) // 2, (y1 + y2) // 2  # Dikdörtgenin ortası
        cx,cy = x1 + w // 2, y1 
        

        cv2.circle(frame, (bx,by),2, (0, 200, 200), -1)  
          
          
        cv2.putText(frame, f'fps : {fps}', (50,50), 3, 1, (255, 0, 0))




        if lim[0] < bx < lim[2] and lim[1]- 10< by < lim[3] +15:
            if cars.count(id)==0 or len(cars)==0:
                cars.append(id)
                if id not in car_start_times:
                    start_time = time.time()
                    car_start_times[id] = start_time
              
                
        if lim4[0]< bx < lim4[2] and lim4[1] - 10< by < lim4[3] + 15:
            if id in car_start_times:
                current_time = time.time()
                time_diff = current_time - car_start_times[id]
                if time_diff > 0:
                    speed = (mesafe * 3.6) / (time_diff)
                    
                    cars_speed.append((id, str(speed)))
                    
                    

        
        
                    
        
        if lim2[0]< bx <lim2[2] and lim2[1]- 10 <by< lim2[3]+10:
            if cars1.count(id)==0 or len(cars1)==0:
                cars1.append(id)
                if id not in car_start_times1:
                    start_time1 = time.time()
                    car_start_times1[id] = start_time1
            
        if lim5[0]< bx< lim5[2] and lim5[1] - 10< by < lim5[3]+10:
            if id in car_start_times1:
                current_time1 = time.time()
                time_diff1 = current_time1 - car_start_times1[id]
                if time_diff1 > 0:
                    speed1 = (mesafe * 3.6) / (time_diff1)
                    
                    cars_speed1.append((id, str(speed1)))
                    
                    
                    

        



        

        if lim3[0]< bx <lim3[2] and lim3[1]  <by< lim3[3] :
            if cars2.count(id)==0 or len(cars2)==0:
                cars2.append(id)
                if id not in car_start_times2:
                    start_time2 = time.time()
                    car_start_times2[id] = start_time2
            
        if lim6[0]< bx < lim6[2] and lim6[1]  < by < lim6[3]:
            if id in car_start_times2:
                current_time2 = time.time()
                time_diff2 = current_time2 - car_start_times2[id]
                if time_diff2 > 0:
                    speed2 = (mesafe * 3.6) / (time_diff2)
                    
                    cars_speed2.append((id, str(speed2)))
                    
                    


        for car_id, car_speed in cars_speed :
            if car_id == id and car_speed == str(speed):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cvzone.putTextRect(frame, f'hiz: {speed:.2f} km/saat', (cx, cy), 1, 2, (255, 255, 255), (255, 100, 0))

        
        for car_id1, car_speed1 in cars_speed1:
            if car_id1 == id and car_speed1 == str(speed1):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cvzone.putTextRect(frame, f'hiz: {speed1:.2f} km/saat', (cx, cy), 1, 2, (255, 255, 255), (255, 100, 0))

                    

        for car_id2, car_speed2 in cars_speed2:
            if car_id2 == id and car_speed2 == str(speed2):
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cvzone.putTextRect(frame, f'hiz: {speed2:.2f} km/saat', (cx, cy), 1, 2, (255, 255, 255), (255, 100, 0))


   
    cv2.imshow("screen",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
out.release()
cv2.destroyAllWindows()
cap.release()


0: 384x640 6 cars, 46.7ms
Speed: 0.0ms preprocess, 46.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 39.8ms
Speed: 1.0ms preprocess, 39.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 24.9ms
Speed: 1.0ms preprocess, 24.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 32.8ms
Speed: 1.0ms preprocess, 32.8ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 31.9ms
Speed: 1.0ms preprocess, 31.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 31.4ms
Speed: 0.0ms preprocess, 31.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 32.1ms
Speed: 1.0ms preprocess, 32.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 5 cars, 44.6ms
Speed: 0.0ms preprocess, 44.6ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x

0